In [33]:
import numpy as np
from astropy.table import Table
import astropy.units as u
import pickle

In [34]:
tall = Table.read('../data/stream_fits_5d.fits')
N = len(tall)

In [35]:
tall

name,rperi,rapo,ecc,vcirc,flag,ra0,pbest [5],wangle,n_steps,"dist [3,2]","phi2 [3,2]","pmdec [3,2]","pmra [3,2]"
bytes17,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64
ATLAS,18.946892524136683,25.183807001929445,0.141328248697007,209.85472576643488,0,30.7,-33.19836601015188 .. 79.74066893084046,360.0,110.0,9.3 .. 4.58,9.3 .. 0.24,9.3 .. 1.0,9.3 .. 1.0
Aliqa Uma,16.35895214344011,31.375316089781414,0.3145824687826762,205.33888933064844,0,40.6,-38.29747811351786 .. 113.99956026106726,360.0,72.0,31.7 .. 5.76,31.7 .. 0.26,31.7 .. 1.0,31.7 .. 1.0
Chenab,27.948605371222772,34.792493880136306,0.10908142494435628,203.05946849265146,0,-28.3,-42.99032463173399 .. 23.640750556849373,180.0,144.0,-40.7 .. 7.96,-40.7 .. 0.71,-40.7 .. 1.0,-40.7 .. 1.0
Elqui,31.798060640200823,50.241125987915964,0.22480799853998382,193.90342885845553,0,20.6,-42.400319589167374 .. 82.2255544662121,360.0,108.0,10.7 .. 10.02,10.7 .. 0.54,10.7 .. 1.0,10.7 .. 1.0
Fimbulthul,1.4272973960687623,7.113636193471706,0.6657748521035963,233.43842527685666,0,214.23,-22.617221350877895 .. 74.9309488747916,360.0,45.0,198.74 .. 0.01,198.74 .. 0.5,198.74 .. 1.0,198.74 .. 1.0
"Fj\""{o}rm",8.181312029887275,23.360416163956753,0.48123882245082467,211.32221725128323,0,197.37,5.530983230813971 .. -111.83822995880149,360.0,207.0,197.37 .. 0.07,197.37 .. 0.5,197.37 .. 1.0,197.37 .. 1.0
"Gj\""{o}ll",9.242344316286504,29.229667706713073,0.5195289338773761,206.83814294458787,0,70.16,-2.5170516281065214 .. -25.408093423299785,360.0,78.0,70.16 .. 0.1,70.16 .. 0.5,70.16 .. 1.0,70.16 .. 1.0
Indus,11.41834321762615,20.83030745296318,0.29185606342037473,213.5160943135424,0,-36.3,-50.670516375709234 .. 1.078408065651399,180.0,60.0,-36.3 .. 3.32,-36.3 .. 0.83,-36.3 .. 1.0,-36.3 .. 1.0
Leiptr,12.731595167982285,43.64941402161866,0.5483729237563725,197.61652251451048,0,61.03,0.7337272822155927 .. -83.3338223470733,360.0,144.0,61.03 .. 0.42,61.03 .. 0.5,61.03 .. 1.0,61.03 .. 1.0


In [38]:
dt = 0.5 * u.Myr

def ln_likelihood(p, ra_0, data, n_steps, dt, wangle):
    # initial conditions at ra_0
    dec, d, pm1, pm2, vr = p
    
    if (d<0) | (np.abs(vr)>500) | (dec<-90) | (dec>90):
        return -np.inf
    
    _phi2_sigma = 0. # deg
    _dist_sigma = 0. # kpc
    _vr_sigma = 0 # km/s
    _pm_sigma = 0 # mas/yr
    
    wdeg = wangle.to(u.deg).value
    
    c = coord.ICRS(ra=ra_0*u.deg, dec=dec*u.deg, distance=d*u.kpc, 
               pm_ra_cosdec=pm1*u.mas/u.yr,
               pm_dec=pm2*u.mas/u.yr,
               radial_velocity=vr*u.km/u.s)

    w0 = gd.PhaseSpacePosition(c.transform_to(gc_frame).cartesian)
    if ham.energy(w0)>0:
        return -np.inf
    
    orbit = ham.integrate_orbit(w0, dt=dt, n_steps=n_steps)
    model_stream = orbit.to_coord_frame(coord.ICRS, galactocentric_frame=gc_frame)
    model_x = model_stream.ra.wrap_at(wangle).degree
    if model_x[-1] < wdeg - 360:
        return -np.inf
    
    model_phi2 = model_stream.dec.degree
    model_dist = model_stream.distance.to(u.kpc).value
    model_pmra = model_stream.pm_ra_cosdec.to(u.mas/u.yr).value
    model_pmdec = model_stream.pm_dec.to(u.mas/u.yr).value
    ix = np.argsort(model_x)
    model_x = model_x[ix]
    
    # define interpolating functions
    order = 3
    bbox = [wdeg - 360, wdeg]
    chi2 = 0
    
    phi2_interp = InterpolatedUnivariateSpline(model_x, model_phi2[ix], k=order, bbox=bbox)
    dist_interp = InterpolatedUnivariateSpline(model_x, model_dist[ix], k=order, bbox=bbox)
    pmra_interp = InterpolatedUnivariateSpline(model_x, model_pmra[ix], k=order, bbox=bbox)
    pmdec_interp = InterpolatedUnivariateSpline(model_x, model_pmdec[ix], k=order, bbox=bbox)
  
    phi2_sigma = np.sqrt(_phi2_sigma**2 + data['phi2'][2]**2)
    chi2 += np.sum(-(phi2_interp(data['phi2'][0]) - data['phi2'][1])**2 / phi2_sigma**2 - 2*np.log(phi2_sigma))
    
    dist_sigma = np.sqrt(_dist_sigma**2 + data['dist'][2]**2)
    chi2 += np.sum(-(dist_interp(data['dist'][0]) - data['dist'][1])**2 / dist_sigma**2 - 2*np.log(dist_sigma))
    
    pmra_sigma = np.sqrt(_pm_sigma**2 + data['pmra'][2]**2)
    chi2 += np.sum(-(pmra_interp(data['pmra'][0]) - data['pmra'][1])**2 / pmra_sigma**2 - 2*np.log(pmra_sigma))
    
    pmdec_sigma = np.sqrt(_pm_sigma**2 + data['pmdec'][2]**2)
    chi2 += np.sum(-(pmdec_interp(data['pmdec'][0]) - data['pmdec'][1])**2 / pmdec_sigma**2 - 2*np.log(pmdec_sigma))
    
    return chi2

In [47]:
percentiles = [16, 50, 84]
p_fit = np.empty((N,3,5))
r_fit = np.empty((N,3,5))

for i in range(N):
    sampler = pickle.load(open('../data/sampler_{:s}.pkl'.format(tall['name'][i]), 'rb'))
    chain = sampler.chain[:,256:,:]
    flatchain = np.reshape(chain,(-1,5))
    
    p_fit[i] = np.percentile(flatchain, percentiles, axis=0)

r_fit[:,0,:] = p_fit[:,1,:]
r_fit[:,1,:] = p_fit[:,2,:] - p_fit[:,1,:]
r_fit[:,2,:] = p_fit[:,1,:] - p_fit[:,0,:]

In [53]:
p_fit[0,:,4], r_fit[0,:,4], p_fit[0,2,4] - p_fit[0,1,4], p_fit[0,1,4] - p_fit[0,0,4], 

(array([ -0.20183762,  96.70981046, 184.41399165]),
 array([96.70981046, 87.70418119, 96.91164808]),
 87.70418119140184,
 96.91164808198741)

In [48]:
p_rp = np.empty((N,3))
p_ra = np.empty((N,3))
r_rp = np.empty((N,3))
r_ra = np.empty((N,3))

for i in range(N):
    t = Table.read('../data/orbit_props_{:s}.fits'.format(tall['name'][i]))
    ind_finite = np.isfinite(t['rperi']) & np.isfinite(t['rapo'])
    t = t[ind_finite]
    p_rp[i] = np.percentile(t['rperi'], percentiles)
    p_ra[i] = np.percentile(t['rapo'], percentiles)
    
r_rp[:,0] = p_rp[:,1]
r_rp[:,1] = p_rp[:,2] - p_rp[:,1]
r_rp[:,2] = p_rp[:,1] - p_rp[:,0]

r_ra[:,0] = p_ra[:,1]
r_ra[:,1] = p_ra[:,2] - p_ra[:,1]
r_ra[:,2] = p_ra[:,1] - p_ra[:,0]

In [55]:
f = open('../paper/table_properties.tex', 'w')
for i in range(N):
    f.write('{:s} & ${:.1f}$ & ${:.1f}^{{+{:.1f}}}_{{-{:.1f}}}$ & ${:.1f}^{{+{:.1f}}}_{{-{:.1f}}}$ & ${:.1f}^{{+{:.1f}}}_{{-{:.1f}}}$ & ${:.1f}^{{+{:.1f}}}_{{-{:.1f}}}$ & ${:.1f}^{{+{:.1f}}}_{{-{:.1f}}}$ & '.format(
        tall['name'][i], tall['ra0'][i], r_fit[i,0,0], r_fit[i,1,0], r_fit[i,2,0], r_fit[i,0,1], r_fit[i,1,1], r_fit[i,2,1],
    r_fit[i,0,2], r_fit[i,1,2], r_fit[i,2,2], r_fit[i,0,3], r_fit[i,1,3], r_fit[i,2,3], r_fit[i,0,4], r_fit[i,1,4], r_fit[i,2,4]))
    f.write('${:.1f}^{{+{:.1f}}}_{{-{:.1f}}}$ & ${:.1f}^{{+{:.1f}}}_{{-{:.1f}}}$ \\\\ \n'.format(
        r_rp[i][0], r_rp[i][1], r_rp[i][2], r_ra[i][0], r_ra[i][1], r_ra[i][2]))
f.close()